In [1]:
# sphinx_gallery_thumbnail_number = 2

# Authors: Robert Luke <mail@robertluke.net>
#
# License: BSD (3-clause)

# Import common libraries
from collections import defaultdict
from copy import deepcopy
from itertools import compress
from pprint import pprint

# Import Plotting Library
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os
import pywt


# Import StatsModels
import statsmodels.formula.api as smf
from mne import Epochs, events_from_annotations, set_log_level
from mne.preprocessing.nirs import (
    beer_lambert_law,
    optical_density,
    scalp_coupling_index,
    temporal_derivative_distribution_repair,
    
)

# Import MNE processing
from mne.viz import plot_compare_evokeds

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids


# Import MNE-NIRS processing
from mne_nirs.channels import get_long_channels, picks_pair_to_idx
from mne_nirs.datasets import fnirs_motor_group
from mne_nirs.signal_enhancement import (enhance_negative_correlation, short_channel_regression)
from mne_nirs.channels import (get_long_channels,
                               get_short_channels,
                               picks_pair_to_idx)

from collections import defaultdict
import numpy as np
from itertools import compress
from sklearn.decomposition import PCA
import mne


# Set general parameters
set_log_level("WARNING")  # Don't show info, as it is repetitive for many subjects

tmin=-3
tmax=14

In [2]:
# cropping the signal before sci calculation
def individual_analysis(bids_path):
    # Read data with annotations in BIDS format
    raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)
    # check if coordinates of the channels
    
    
    #print(raw_intensity.ch_names)

    
    # Get event timings
    #print("Extracting event timings...")
    Breaks, _ = mne.events_from_annotations(raw_intensity, {'Xstart': 4, 'Xend': 5})
    AllEvents, _ = mne.events_from_annotations(raw_intensity)
    Breaks = Breaks[:, 0] / raw_intensity.info['sfreq']
    LastEvent = AllEvents[-1, 0] / raw_intensity.info['sfreq']
    
    if len(Breaks) % 2 == 0:
        raise ValueError("Breaks array should have an odd number of elements.")
    
    original_duration = raw_intensity.times[-1] - raw_intensity.times[0]
    #print(f"Original duration: {original_duration:.2f} seconds")
    
    # Cropping dataset
    #print("Cropping the dataset...")
    cropped_intensity = raw_intensity.copy().crop(Breaks[0], Breaks[1])
    for j in range(2, len(Breaks) - 1, 2):
        block = raw_intensity.copy().crop(Breaks[j], Breaks[j + 1])
        cropped_intensity.append(block)
    cropped_intensity.append(raw_intensity.copy().crop(Breaks[-1], LastEvent + 15.25))
    
    cropped_duration = cropped_intensity.times[-1] - cropped_intensity.times[0]
    #print(f"Cropped duration: {cropped_duration:.2f} seconds")
    
    if cropped_duration >= original_duration:
        print(f"WARNING: Cropping did not reduce duration!")
    
    raw_intensity_cropped = cropped_intensity.copy()

    

    
    # Remove break annotations
    #print("Removing break annotations for the orginal raw...")
    raw_intensity.annotations.delete(np.where(
        (raw_intensity.annotations.description == 'Xstart') | 
        (raw_intensity.annotations.description == 'Xend') | 
        (raw_intensity.annotations.description == 'BAD boundary') | 
        (raw_intensity.annotations.description == 'EDGE boundary')
    )[0])
    
    #print("Removing break annotations for the cropped raw...")
    raw_intensity_cropped.annotations.delete(np.where(
        (raw_intensity_cropped.annotations.description == 'Xstart') | 
        (raw_intensity_cropped.annotations.description == 'Xend') | 
        (raw_intensity_cropped.annotations.description == 'BAD boundary') | 
        (raw_intensity_cropped.annotations.description == 'EDGE boundary')
    )[0]) 
    
    
    
    # Convert signal to optical density and determine bad channels
    raw_od = optical_density(raw_intensity)
    raw_od_cropped = optical_density(raw_intensity_cropped)
    
    # get the total number of short channels
    short_chs = get_short_channels(raw_od)
    tot_number_of_short_channels = len(short_chs.ch_names)
    

    # sci calculated
    sci = scalp_coupling_index(raw_od_cropped, l_freq=0.7, h_freq=1.45)
    #sci = scalp_coupling_index(raw_od_cropped, h_freq=1.35)
    bad_channels= list(compress(raw_od.ch_names, sci < 0.8))
    
    if len(bad_channels) > 55:
        print(f"❌ Too many bad channels ({len(bad_channels)}). Excluding subject from analysis.")
        return None, None, None, None, None
    
    raw_od.info["bads"] = bad_channels
    raw_intensity_cropped.info["bads"] = bad_channels
    
    #print(f"Bad channels: {raw_od.info['bads']}")
    # print the number of bad channels
    #print(f"Number of bad channels: {len(raw_od.info['bads'])}")
    
    # Remove bad channels
   
    """ raw_od.drop_channels(bad_channels)
    raw_intensity_cropped.drop_channels(bad_channels) """
    
    raw_od = temporal_derivative_distribution_repair(raw_od)
    raw_od_cropped = temporal_derivative_distribution_repair(raw_od_cropped)

    
     # Get long channels
    long_chs = get_long_channels(raw_od)
    bad_long_chs = long_chs.info["bads"]
    
    """ print(f"Number of all (good and bad) short channels: {tot_number_of_short_channels}")
    print(f"Number of bad long channels: {len(bad_long_chs)}")
    print(f"Number of long and short bad channels: {len(bad_channels)}") """
    # print the number of short bad channels
    len_bad_short_chs = len(bad_channels) - len(bad_long_chs)
    num_good_short_channels = tot_number_of_short_channels - len_bad_short_chs
    # Print diagnostics
    """ print(f"Number of all (good and bad) short channels: {tot_number_of_short_channels}")
    print(f"Number of bad long channels: {len(bad_long_chs)}")
    print(f"Number of bad short channels: {len_bad_short_chs}")
    print(f"✅ Number of good short channels: {num_good_short_channels}") """
    #print(f"Number of bad short channels: {len_bad_short_chs}")
    
    # Determine if there are short channels
    if num_good_short_channels < 4:
        print("❌ No short channels found. Skipping the subject.")
        return None, None, None, None, None # Keep the data unchanged
    else:
        #print("Applying short-channel regression.")
        raw_od_corrected = short_channel_regression(raw_od)
        #raw_od_corrected=raw_od.copy()
        # drop the bad channels
        #raw_od_corrected.drop_channels(bad_channels)
        
        # interpolate the bad channels
        #raw_od_corrected.interpolate_bads()
        
    # short-channel regression subtracts a scaled version of the signal obtained from the nearest short channel from the signal obtained from the long channel. 

    raw_haemo = beer_lambert_law(raw_od_corrected, ppf=0.1)
    
    #raw_haemo = get_long_channels(raw_haemo, min_dist=0.02, max_dist=0.04) # max_dist 40mm
    raw_haemo = get_long_channels(raw_haemo, min_dist=0.02) 
    
    
   
    

    # Convert to haemoglobin and filter
    
    # check the ppf
    
    #raw_haemo.plot_psd(average=True, show=True)
    
    """ raw_haemo = raw_haemo.filter(
    l_freq=0.01, h_freq=0.7, method="fir", fir_design="firwin", verbose=False,
    h_trans_bandwidth=0.3, l_trans_bandwidth=0.005) """
    
    # improved filter
    """ raw_haemo = raw_haemo.filter(
    l_freq=0.05, h_freq=0.2, method="fir", fir_design="firwin", verbose=False,
    h_trans_bandwidth=0.01, l_trans_bandwidth=0.01) """
    
    #raw_haemo.plot_psd(average=True, show=True)

    raw_haemo = raw_haemo.filter(l_freq = None, h_freq = 0.2,  
                                 method="iir", iir_params =dict(order=5, ftype='butter'))
     #high-pass
    raw_haemo= raw_haemo.filter(l_freq =  0.05, h_freq = None, method="iir", iir_params =dict(order=5, ftype='butter')) #t0.05 was cutoff in andreas analysis
      
    """ # drop the bad channels
    raw_heamo_good = raw_haemo.copy()
    
    raw_heamo_good.drop_channels(raw_haemo.info['bads']) """
    
    
    #raw_haemo.plot(n_channels=len(raw_haemo.ch_names), duration=5000,show_scrollbars=True )

    # Extract events but ignore those with
    # the word Ends (i.e. drop ExperimentEnds events)
    events, event_dict = events_from_annotations(
        raw_haemo, verbose=False, regexp="^(?![Ends]).*$"
    )
    epochs = Epochs(
        raw_haemo,
        events,
        event_id=event_dict,
        tmin=-3,
        tmax=14,
        reject=dict(hbo=100e-6),
        reject_by_annotation=True,
        proj=True,
        baseline=(None, 0),
        detrend=1,
        preload=True,
        verbose=None,
    )
    """ epochs['Control'].plot()
    epochs['Noise'].plot()
    epochs['Speech'].plot()
     """


    return raw_haemo, epochs, event_dict, raw_od, events

# Individual participant's mean+std dev

In [3]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from mne_bids import BIDSPath

# === Paths ===
bids_root = r"C:\Datasets\Test-retest study\bids_dataset"
output_root = "individual_analysis_outputs"
os.makedirs(output_root, exist_ok=True)

# === List subjects ===
subject_list = sorted([d for d in os.listdir(bids_root) if d.startswith("sub-")])
subject_list = [s.replace("sub-", "") for s in subject_list]
print("Detected subjects:", subject_list)

# === Storage ===
summary_stats = []

# === Loop over subjects and sessions ===
for sub in subject_list:
    for ses in range(1, 3):
        bids_path = BIDSPath(
            subject=f"{sub}",
            session=f"{ses:02d}",
            task="auditory",
            datatype="nirs",
            root=bids_root,
            suffix="nirs",
            extension=".snirf",
        )

        raw_haemo, epochs, event_dict, raw_od, events = individual_analysis(bids_path)

        if raw_haemo is None or len(epochs) < 10:
            print(f"⚠️ Skipping sub-{sub} ses-{ses:02d} (not enough channels or epochs)")
            continue

        # Output directory
        output_dir = os.path.join(output_root, f"sub-{sub}", f"ses-{ses:02d}")
        os.makedirs(output_dir, exist_ok=True)

        # Drop bad channels
        epochs.drop_channels(epochs.info['bads'])

        # Pick only HbO
        hbo_epochs = epochs.copy().pick("hbo")
        #hbo_epochs.crop(tmin=0.0, tmax=10.0)

        for condition in hbo_epochs.event_id:
            condition_epochs = hbo_epochs[condition]

            if len(condition_epochs) < 3:
                print(f"⚠️ Skipping {condition} for sub-{sub} ses-{ses:02d} (too few epochs)")
                continue

            data = condition_epochs.get_data()  # shape: (n_epochs, n_channels, n_times)

            # Compute average amplitude across all channels and time points, per epoch
            epoch_means = np.mean(data, axis=(1, 2))  # shape: (n_epochs,)

            # Compute mean and std dev across epochs
            mean_all = np.mean(epoch_means)
            std_all = np.std(epoch_means, ddof=1)

            summary_stats.append({
                "Subject": f"sub-{sub}",
                "Session": f"ses-{ses:02d}",
                "Condition": condition,
                "Mean_Amplitude": mean_all,
                "SD_Amplitude": std_all,
                "Num_Epochs": len(condition_epochs)
            })

        print(f"✅ Finished sub-{sub} ses-{ses:02d}")

# === Save summary stats ===
stats_df = pd.DataFrame(summary_stats)

# save to csv
stats_df.to_csv(os.path.join(output_root, "individual_mean_stddev_zscore.csv"), index=False)



Detected subjects: ['01', '02', '03', '04', '05', '07', '08', '10', '11', '12', '13', '16', '17', '19', '21', '24']
✅ Finished sub-01 ses-01
✅ Finished sub-01 ses-02
✅ Finished sub-02 ses-01
✅ Finished sub-02 ses-02
✅ Finished sub-03 ses-01
✅ Finished sub-03 ses-02
✅ Finished sub-04 ses-01
✅ Finished sub-04 ses-02
✅ Finished sub-05 ses-01
✅ Finished sub-05 ses-02
✅ Finished sub-07 ses-01
✅ Finished sub-07 ses-02
✅ Finished sub-08 ses-01
✅ Finished sub-08 ses-02
✅ Finished sub-10 ses-01
✅ Finished sub-10 ses-02
✅ Finished sub-11 ses-01
✅ Finished sub-11 ses-02
✅ Finished sub-12 ses-01
✅ Finished sub-12 ses-02
✅ Finished sub-13 ses-01
✅ Finished sub-13 ses-02
✅ Finished sub-16 ses-01
✅ Finished sub-16 ses-02
✅ Finished sub-17 ses-01
✅ Finished sub-17 ses-02
✅ Finished sub-19 ses-01
✅ Finished sub-19 ses-02
✅ Finished sub-21 ses-01
✅ Finished sub-21 ses-02
✅ Finished sub-24 ses-01
✅ Finished sub-24 ses-02


# Individual mean + std dev rejection

In [4]:
def reject_epochs_zscore_individual(data, subject, session, condition_name, stats_df, z_thresh=3.0, max_reject_ratio=1.0):
    """
    Rejects epochs based on z-score threshold using subject/session/condition-specific stats from stats_df.

    Parameters:
        data (ndarray): (n_epochs, n_times) or (n_epochs,) averaged data per epoch
        subject (str): e.g., "sub-01"
        session (str): e.g., "ses-01"
        condition_name (str): "Control", "Noise", or "Speech"
        stats_df (pd.DataFrame): DataFrame with per-participant mean and std
        z_thresh (float): Z-score threshold
        max_reject_ratio (float): Max proportion of epochs to reject

    Returns:
        cleaned_data (ndarray), rejected_indices (list)
    """
    row = stats_df[
        (stats_df["Subject"] == f"sub-{subject}") &
        (stats_df["Session"] == f"ses-{session}") &
        (stats_df["Condition"] == condition_name)
    ]

    if row.empty:
        raise ValueError(f"No stats found for {subject}, {session}, {condition_name}")

    mean = row["Mean_Amplitude"].values[0]
    std = row["SD_Amplitude"].values[0]

    z_scores = (data - mean) / std
    reject_mask = np.abs(z_scores) > z_thresh
    rejected_indices = np.where(reject_mask)[0].tolist()

    # Limit rejections
    max_reject = int(max_reject_ratio * data.shape[0])
    if len(rejected_indices) > max_reject:
        rejected_indices = rejected_indices[:max_reject]

    cleaned_data = np.delete(data, rejected_indices, axis=0)
    return cleaned_data, rejected_indices


In [5]:
from scipy.stats import ttest_ind
# === Setup ===
output_root = "individual_analysis_outputs"
os.makedirs(output_root, exist_ok=True)

bids_root = r"C:\\Datasets\\Test-retest study\\bids_dataset"
subject_list = sorted([d for d in os.listdir(bids_root) if d.startswith("sub-")])
subject_list = [s.replace("sub-", "") for s in subject_list]

print("Detected subjects:", subject_list)

ttest_results = []


for sub in subject_list:
    for ses in range(1, 3):
        bids_path = BIDSPath(
            subject=f"{sub}",
            session=f"{ses:02d}",
            task="auditory",
            datatype="nirs",
            root=bids_root,
            suffix="nirs",
            extension=".snirf",
        )
        print(f"Processing Subject {sub}, Session {ses:02d}")

        raw_haemo, epochs, event_dict, raw_od, events = individual_analysis(bids_path)

        if raw_haemo is None or len(epochs) < 10:
            print(f"⚠️ Skipping Subject {sub}, Session {ses:02d} (insufficient data)")
            continue

        epochs.drop_channels(epochs.info['bads'])
        epochs_cleaned = epochs.copy().pick("hbo")
        #epochs_cleaned.crop(tmin=0.0, tmax=10.0)

        # Get data and labels
        data = epochs_cleaned.get_data().mean(axis=1)  # (n_epochs, n_channels, n_times)
        data_mean = data.mean(axis=1)
        ev = epochs_cleaned.events[:, 2]
        index_column = np.arange(0, len(ev)).reshape(-1, 1)
        labels = np.hstack((index_column, ev.reshape(-1, 1)))

        # Reject epochs per condition
        conditions = {1: "Control", 2: "Noise", 3: "Speech"}
        all_rejected = []
        for cond_code, cond_name in conditions.items():
            cond_idx = labels[labels[:, 1] == cond_code][:, 0].astype(int)
            cond_data = data_mean[cond_idx]

            try:
                cleaned, rejected = reject_epochs_zscore_individual(
                    cond_data,
                    subject=sub,
                    session=f"{ses:02d}",
                    condition_name=cond_name,
                    stats_df=stats_df,
                    z_thresh=2
                )
            except ValueError as e:
                print(f"⚠️ {e}")
                continue

            rejected_indices = [cond_idx[i] for i in rejected]
            all_rejected.extend(rejected_indices)
            print(f"Subject {sub}, Session {ses:02d} - Rejected {len(rejected)} epochs in {cond_name}")

        # Drop all rejected epochs
        epochs_cleaned.drop(all_rejected)

        # === ROI definitions ===
        roi_definitions = {
            "Noise_ROI": ["S10_D11 hbo", "S5_D3 hbo", "S11_D11 hbo", "S10_D9 hbo", "S10_D12 hbo", "S10_D10 hbo"],
            "Speech_ROI": ["S10_D11 hbo", "S10_D10 hbo", "S10_D12 hbo", "S11_D12 hbo", "S1_D1 hbo", "S10_D9 hbo", "S8_D7 hbo", "S7_D7 hbo", "S8_D8 hbo"],
            "Common_ROI": ["S10_D11 hbo", "S10_D9 hbo", "S10_D12 hbo", "S10_D10 hbo"],
            "Only_Noise_ROI": ["S5_D3 hbo", "S11_D11 hbo"],
            "Only_Speech_ROI": ["S11_D12 hbo", "S1_D1 hbo", "S8_D7 hbo", "S7_D7 hbo", "S8_D8 hbo"],
            "Left_Auditory": ["S4_D2 hbo", "S4_D3 hbo", "S5_D2 hbo", "S5_D3 hbo", "S5_D4 hbo", "S5_D5 hbo"],
            "Right_Auditory": ["S10_D9 hbo", "S10_D10 hbo", "S10_D11 hbo", "S10_D12 hbo", "S11_D11 hbo", "S11_D12 hbo"],
            "Visual": ["S6_D6 hbo", "S6_D8 hbo", "S7_D6 hbo", "S7_D7 hbo", "S8_D7 hbo", "S8_D8 hbo", "S9_D8 hbo", "S7_D8 hbo"],
            "Front": ["S1_D1 hbo", "S2_D1 hbo", "S3_D1 hbo", "S3_D2 hbo", "S12_D1 hbo"]
        }

        tmin, tmax = 4, 6
        for roi_name, optode_list in roi_definitions.items():
            available_optodes = [opt for opt in optode_list if opt in epochs_cleaned.info["ch_names"]]
            if len(available_optodes) < 2:
                continue

            try:
                cond_data = {}
                for cond in ["Control", "Noise", "Speech"]:
                    cond_epochs = epochs_cleaned[cond].copy().pick(available_optodes).crop(tmin=tmin, tmax=tmax)
                    data = cond_epochs.get_data()
                    cond_data[cond] = data.mean(axis=1).mean(axis=1) * 1e6

                if len(cond_data["Control"]) > 1 and len(cond_data["Noise"]) > 1:
                    t_stat, p_val = ttest_ind(cond_data["Control"], cond_data["Noise"], equal_var=False, alternative='less')
                    effect = np.mean(cond_data["Noise"]) - np.mean(cond_data["Control"])
                    ttest_results.append({"Subject": sub, "Session": f"{ses:02d}", "ROI": roi_name, "Comparison": "Control vs Noise", "t_stat": t_stat, "p_value": p_val, "Effect": effect})

                if len(cond_data["Control"]) > 1 and len(cond_data["Speech"]) > 1:
                    t_stat, p_val = ttest_ind(cond_data["Control"], cond_data["Speech"], equal_var=False, alternative='less')
                    effect = np.mean(cond_data["Speech"]) - np.mean(cond_data["Control"])
                    ttest_results.append({"Subject": sub, "Session": f"{ses:02d}", "ROI": roi_name, "Comparison": "Control vs Speech", "t_stat": t_stat, "p_value": p_val, "Effect": effect})

            except Exception as e:
                print(f"Error testing {roi_name} for Subject {sub}, Session {ses:02d}: {e}")

# === Save results ===
df_ttest = pd.DataFrame(ttest_results)

# === Summary per participant ===
print("\n=== Significant ROI Summary per Participant ===")
df_significant = df_ttest[df_ttest["p_value"] < 0.05]
if df_significant.empty:
    print("No significant ROIs found.")
else:
    grouped = df_significant.groupby(["Subject", "Session"])
    for (sub, ses), group in grouped:
        print(f"\nSubject: {sub}, Session: {ses}")
        for _, row in group.iterrows():
            sign = "↑" if row["Effect"] > 0 else "↓"
            print(f"  - {row['ROI']} ({row['Comparison']}): p = {row['p_value']:.4f}, effect = {row['Effect']:.4f} ({sign})")

print("\n✅ Done with ROI t-tests!")

Detected subjects: ['01', '02', '03', '04', '05', '07', '08', '10', '11', '12', '13', '16', '17', '19', '21', '24']
Processing Subject 01, Session 01
Subject 01, Session 01 - Rejected 2 epochs in Control
Subject 01, Session 01 - Rejected 1 epochs in Noise
Subject 01, Session 01 - Rejected 1 epochs in Speech
Processing Subject 01, Session 02
Subject 01, Session 02 - Rejected 2 epochs in Control
Subject 01, Session 02 - Rejected 1 epochs in Noise
Subject 01, Session 02 - Rejected 1 epochs in Speech
Processing Subject 02, Session 01
Subject 02, Session 01 - Rejected 1 epochs in Control
Subject 02, Session 01 - Rejected 1 epochs in Noise
Subject 02, Session 01 - Rejected 0 epochs in Speech
Processing Subject 02, Session 02
Subject 02, Session 02 - Rejected 1 epochs in Control
Subject 02, Session 02 - Rejected 1 epochs in Noise
Subject 02, Session 02 - Rejected 0 epochs in Speech
Processing Subject 03, Session 01
Subject 03, Session 01 - Rejected 0 epochs in Control
Subject 03, Session 01 -

In [6]:
from scipy.stats import ttest_ind
# === Setup ===
output_root = "individual_analysis_outputs"
os.makedirs(output_root, exist_ok=True)

bids_root = r"C:\\Datasets\\Test-retest study\\bids_dataset"
subject_list = sorted([d for d in os.listdir(bids_root) if d.startswith("sub-")])
subject_list = [s.replace("sub-", "") for s in subject_list]

print("Detected subjects:", subject_list)

ttest_results = []


for sub in subject_list:
    for ses in range(1, 3):
        bids_path = BIDSPath(
            subject=f"{sub}",
            session=f"{ses:02d}",
            task="auditory",
            datatype="nirs",
            root=bids_root,
            suffix="nirs",
            extension=".snirf",
        )
        print(f"Processing Subject {sub}, Session {ses:02d}")

        raw_haemo, epochs, event_dict, raw_od, events = individual_analysis(bids_path)

        if raw_haemo is None or len(epochs) < 10:
            print(f"⚠️ Skipping Subject {sub}, Session {ses:02d} (insufficient data)")
            continue

        epochs.drop_channels(epochs.info['bads'])
        epochs_cleaned = epochs.copy().pick("hbo")
        #epochs_cleaned.crop(tmin=0.0, tmax=10.0)

        # Get data and labels
        data = epochs_cleaned.get_data().mean(axis=2)  # (n_epochs, n_channels)
        ev = epochs_cleaned.events[:, 2]
        index_column = np.arange(0, len(ev)).reshape(-1, 1)
        labels = np.hstack((index_column, ev.reshape(-1, 1)))

        # Reject epochs per condition
        conditions = {1: "Control", 2: "Noise", 3: "Speech"}
        all_rejected = []

        for cond_code, cond_name in conditions.items():
            cond_idx = labels[labels[:, 1] == cond_code][:, 0].astype(int)
            cond_epochs = epochs_cleaned[cond_idx]  # Filter epochs for this condition

            if len(cond_epochs) == 0:
                continue

            available_channels = cond_epochs.info["ch_names"]
            rejected_epoch_indices = set()

            for ch_idx, ch_name in enumerate(available_channels):
                try:
                    # Get mean time-series for each epoch for this channel
                    ch_data = cond_epochs.get_data()[:, ch_idx, :].mean(axis=1)  # (n_epochs,)
                    _, rejected = reject_epochs_zscore_individual(
                        ch_data,
                        subject=sub,
                        session=f"{ses:02d}",
                        condition_name=cond_name,
                        stats_df=stats_df,
                        z_thresh=3.0
                    )

                    # Map local indices to global indices
                    global_rejected = [cond_idx[i] for i in rejected]
                    rejected_epoch_indices.update(global_rejected)

                except Exception as e:
                    print(f"⚠️ Channel {ch_name} skipped for Subject {sub}, Session {ses:02d}, Condition {cond_name}: {e}")
                    continue

            all_rejected.extend(rejected_epoch_indices)
            print(f"Subject {sub}, Session {ses:02d} - Rejected {len(rejected_epoch_indices)} epochs in {cond_name}")


        # Drop all rejected epochs
        epochs_cleaned.drop(all_rejected)

        # === ROI definitions ===
        roi_definitions = {
            "Noise_ROI": ["S10_D11 hbo", "S5_D3 hbo", "S11_D11 hbo", "S10_D9 hbo", "S10_D12 hbo", "S10_D10 hbo"],
            "Speech_ROI": ["S10_D11 hbo", "S10_D10 hbo", "S10_D12 hbo", "S11_D12 hbo", "S1_D1 hbo", "S10_D9 hbo", "S8_D7 hbo", "S7_D7 hbo", "S8_D8 hbo"],
            "Common_ROI": ["S10_D11 hbo", "S10_D9 hbo", "S10_D12 hbo", "S10_D10 hbo"],
            "Only_Noise_ROI": ["S5_D3 hbo", "S11_D11 hbo"],
            "Only_Speech_ROI": ["S11_D12 hbo", "S1_D1 hbo", "S8_D7 hbo", "S7_D7 hbo", "S8_D8 hbo"],
            "Left_Auditory": ["S4_D2 hbo", "S4_D3 hbo", "S5_D2 hbo", "S5_D3 hbo", "S5_D4 hbo", "S5_D5 hbo"],
            "Right_Auditory": ["S10_D9 hbo", "S10_D10 hbo", "S10_D11 hbo", "S10_D12 hbo", "S11_D11 hbo", "S11_D12 hbo"],
            "Visual": ["S6_D6 hbo", "S6_D8 hbo", "S7_D6 hbo", "S7_D7 hbo", "S8_D7 hbo", "S8_D8 hbo", "S9_D8 hbo", "S7_D8 hbo"],
            "Front": ["S1_D1 hbo", "S2_D1 hbo", "S3_D1 hbo", "S3_D2 hbo", "S12_D1 hbo"]
        }

        tmin, tmax = 4, 6
        for roi_name, optode_list in roi_definitions.items():
            available_optodes = [opt for opt in optode_list if opt in epochs_cleaned.info["ch_names"]]
            if len(available_optodes) < 2:
                continue

            try:
                cond_data = {}
                for cond in ["Control", "Noise", "Speech"]:
                    cond_epochs = epochs_cleaned[cond].copy().pick(available_optodes).crop(tmin=tmin, tmax=tmax)
                    data = cond_epochs.get_data()
                    cond_data[cond] = data.mean(axis=1).mean(axis=1) * 1e6

                if len(cond_data["Control"]) > 1 and len(cond_data["Noise"]) > 1:
                    t_stat, p_val = ttest_ind(cond_data["Control"], cond_data["Noise"], equal_var=False, alternative='less')
                    effect = np.mean(cond_data["Noise"]) - np.mean(cond_data["Control"])
                    ttest_results.append({"Subject": sub, "Session": f"{ses:02d}", "ROI": roi_name, "Comparison": "Control vs Noise", "t_stat": t_stat, "p_value": p_val, "Effect": effect})

                if len(cond_data["Control"]) > 1 and len(cond_data["Speech"]) > 1:
                    t_stat, p_val = ttest_ind(cond_data["Control"], cond_data["Speech"], equal_var=False, alternative='less')
                    effect = np.mean(cond_data["Speech"]) - np.mean(cond_data["Control"])
                    ttest_results.append({"Subject": sub, "Session": f"{ses:02d}", "ROI": roi_name, "Comparison": "Control vs Speech", "t_stat": t_stat, "p_value": p_val, "Effect": effect})

            except Exception as e:
                print(f"Error testing {roi_name} for Subject {sub}, Session {ses:02d}: {e}")

# === Save results ===
df_ttest = pd.DataFrame(ttest_results)

# === Summary per participant ===
print("\n=== Significant ROI Summary per Participant ===")
df_significant = df_ttest[df_ttest["p_value"] < 0.05]
if df_significant.empty:
    print("No significant ROIs found.")
else:
    grouped = df_significant.groupby(["Subject", "Session"])
    for (sub, ses), group in grouped:
        print(f"\nSubject: {sub}, Session: {ses}")
        for _, row in group.iterrows():
            sign = "↑" if row["Effect"] > 0 else "↓"
            print(f"  - {row['ROI']} ({row['Comparison']}): p = {row['p_value']:.4f}, effect = {row['Effect']:.4f} ({sign})")

print("\n✅ Done with ROI t-tests!")

Detected subjects: ['01', '02', '03', '04', '05', '07', '08', '10', '11', '12', '13', '16', '17', '19', '21', '24']
Processing Subject 01, Session 01
Subject 01, Session 01 - Rejected 9 epochs in Control
Subject 01, Session 01 - Rejected 15 epochs in Noise
Subject 01, Session 01 - Rejected 8 epochs in Speech
Processing Subject 01, Session 02
Subject 01, Session 02 - Rejected 7 epochs in Control
Subject 01, Session 02 - Rejected 7 epochs in Noise
Subject 01, Session 02 - Rejected 10 epochs in Speech
Processing Subject 02, Session 01
Subject 02, Session 01 - Rejected 4 epochs in Control
Subject 02, Session 01 - Rejected 4 epochs in Noise
Subject 02, Session 01 - Rejected 4 epochs in Speech
Processing Subject 02, Session 02
Subject 02, Session 02 - Rejected 10 epochs in Control
Subject 02, Session 02 - Rejected 6 epochs in Noise
Subject 02, Session 02 - Rejected 3 epochs in Speech
Processing Subject 03, Session 01
Subject 03, Session 01 - Rejected 7 epochs in Control
Subject 03, Session 0

# Global Mean + Std dev

In [7]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from mne_bids import BIDSPath

# === Paths ===
bids_root = r"C:\Datasets\Test-retest study\bids_dataset"
output_root = "individual_analysis_outputs"
os.makedirs(output_root, exist_ok=True)

# === List subjects ===
subject_list = sorted([d for d in os.listdir(bids_root) if d.startswith("sub-")])
subject_list = [s.replace("sub-", "") for s in subject_list]
print("Detected subjects:", subject_list)

# === Storage ===
condition_timecourses = defaultdict(list)
n_times_map = {}

# === Loop over subjects and sessions ===
for sub in subject_list:
    for ses in range(1, 3):
        bids_path = BIDSPath(
            subject=f"{sub}",
            session=f"{ses:02d}",
            task="auditory",
            datatype="nirs",
            root=bids_root,
            suffix="nirs",
            extension=".snirf",
        )

        raw_haemo, epochs, event_dict, raw_od, events = individual_analysis(bids_path)

        if raw_haemo is None or len(epochs) < 10:
            print(f"⚠️ Skipping sub-{sub} ses-{ses:02d} (not enough channels or epochs)")
            continue

        # Drop bad channels
        epochs.drop_channels(epochs.info['bads'])

        # Pick only HbO
        hbo_epochs = epochs.copy().pick("hbo")
        #hbo_epochs.crop(tmin=0.0, tmax=10.0)

        for condition in hbo_epochs.event_id:
            condition_epochs = hbo_epochs[condition]

            if len(condition_epochs) < 3:
                print(f"⚠️ Skipping {condition} for sub-{sub} ses-{ses:02d} (too few epochs)")
                continue

            data = condition_epochs.get_data()  # (n_epochs, n_channels, n_times)
            avg_timecourse = np.mean(data, axis=(0, 1))  # (n_times,) — avg over epochs and channels

            condition_timecourses[condition].append(avg_timecourse)
            n_times_map[condition] = avg_timecourse.shape[0]

        print(f"✅ Finished sub-{sub} ses-{ses:02d}")

# === Compute mean and std per condition ===
group_level_results = []

for condition, vectors in condition_timecourses.items():
    stacked = np.vstack(vectors)  # shape: (n_participants, n_times)
    flat_data = stacked.flatten()  # Combine all timepoints and participants
    mean_signal = np.mean(flat_data)
    std_signal = np.std(flat_data, ddof=1)

    # Save individual timecourses just in case
    condition_df = pd.DataFrame(stacked)
    

    group_level_results.append({
        "Condition": condition,
        "N_Participants": len(vectors),
        "Global_Mean": mean_signal,
        "Global_Std": std_signal
        
    })

# === Summary Table ===
summary_df = pd.DataFrame(group_level_results)

# save to csv
summary_df.to_csv(os.path.join(output_root, "global_mean_stddev_zscore.csv"), index=False)




Detected subjects: ['01', '02', '03', '04', '05', '07', '08', '10', '11', '12', '13', '16', '17', '19', '21', '24']
✅ Finished sub-01 ses-01
✅ Finished sub-01 ses-02
✅ Finished sub-02 ses-01
✅ Finished sub-02 ses-02
✅ Finished sub-03 ses-01
✅ Finished sub-03 ses-02
✅ Finished sub-04 ses-01
✅ Finished sub-04 ses-02
✅ Finished sub-05 ses-01
✅ Finished sub-05 ses-02
✅ Finished sub-07 ses-01
✅ Finished sub-07 ses-02
✅ Finished sub-08 ses-01
✅ Finished sub-08 ses-02
✅ Finished sub-10 ses-01
✅ Finished sub-10 ses-02
✅ Finished sub-11 ses-01
✅ Finished sub-11 ses-02
✅ Finished sub-12 ses-01
✅ Finished sub-12 ses-02
✅ Finished sub-13 ses-01
✅ Finished sub-13 ses-02
✅ Finished sub-16 ses-01
✅ Finished sub-16 ses-02
✅ Finished sub-17 ses-01
✅ Finished sub-17 ses-02
✅ Finished sub-19 ses-01
✅ Finished sub-19 ses-02
✅ Finished sub-21 ses-01
✅ Finished sub-21 ses-02
✅ Finished sub-24 ses-01
✅ Finished sub-24 ses-02


In [50]:
print(summary_df)

  Condition  N_Participants   Global_Mean  Global_Std
0   Control              32 -1.467584e-07    0.000001
1     Noise              32  6.891795e-07    0.000001
2    Speech              32  9.625289e-07    0.000002


# Epoch rejection function (global mean std dev)

In [8]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, zscore
from mne_bids import BIDSPath
from collections import defaultdict

def reject_epochs_zscore_global(data, condition_name, df= summary_df, z_thresh=3.0, max_reject_ratio=1.0):
    """
    Rejects epochs based on z-score threshold using global mean and std from summary_df.

    Parameters:
        data (ndarray): (n_epochs, n_channels) — max or min per channel per epoch
        condition_name (str): "Control", "Noise", or "Speech"
        summary_df (pd.DataFrame): Contains global mean/std per condition
        z_thresh (float): Z-score threshold for rejection
        max_reject_ratio (float): Max proportion of epochs to reject

    Returns:
        cleaned_data (ndarray), rejected_indices (list)
    """
    row = summary_df[summary_df["Condition"] == condition_name]
    if row.empty:
        raise ValueError(f"No global stats found for condition: {condition_name}")
    
    global_mean = row["Global_Mean"].values[0]
    global_std = row["Global_Std"].values[0]

    z_scores = (data - global_mean) / global_std
    reject_mask = np.abs(z_scores) > z_thresh
    #epoch_reject_flags = np.any(reject_mask)
    rejected_indices = np.where(reject_mask)[0].tolist()

    # Limit number of rejected epochs
    max_reject = int(max_reject_ratio * data.shape[0])
    if len(rejected_indices) > max_reject:
        rejected_indices = rejected_indices[:max_reject]

    cleaned_data = np.delete(data, rejected_indices, axis=0)
    return cleaned_data, rejected_indices


In [9]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, zscore
from mne_bids import BIDSPath
from collections import defaultdict

# === Setup ===
output_root = "individual_analysis_outputs"
os.makedirs(output_root, exist_ok=True)

bids_root = r"C:\\Datasets\\Test-retest study\\bids_dataset"
subject_list = sorted([d for d in os.listdir(bids_root) if d.startswith("sub-")])
subject_list = [s.replace("sub-", "") for s in subject_list]

print("Detected subjects:", subject_list)

ttest_results = []


for sub in subject_list:
    for ses in range(1, 3):
        bids_path = BIDSPath(
            subject=f"{sub}",
            session=f"{ses:02d}",
            task="auditory",
            datatype="nirs",
            root=bids_root,
            suffix="nirs",
            extension=".snirf",
        )
        print(f"Processing Subject {sub}, Session {ses:02d}")

        raw_haemo, epochs, event_dict, raw_od, events = individual_analysis(bids_path)

        if raw_haemo is None or len(epochs) < 10:
            print(f"⚠️ Skipping Subject {sub}, Session {ses:02d} (insufficient data)")
            continue

        epochs.drop_channels(epochs.info['bads'])
        epochs_cleaned = epochs.copy().pick("hbo")
        #epochs_cleaned.crop(tmin=0.0, tmax=10.0)

        # Get data and labels
        data = epochs_cleaned.get_data().mean(axis=1) # average over channels (n_epochs, n_times)
        data_mean = data.mean(axis=1) # average over time
        ev = epochs_cleaned.events[:, 2]
        index_column = np.arange(0, len(ev)).reshape(-1, 1)
        labels = np.hstack((index_column, ev.reshape(-1, 1)))

        # Reject epochs per condition
        conditions = {1: "Control", 2: "Noise", 3: "Speech"}
        all_rejected = []
        for cond_code, cond_name in conditions.items():
            cond_idx = labels[labels[:, 1] == cond_code][:, 0].astype(int)
            cond_data = data_mean[cond_idx]
            a, rejected = reject_epochs_zscore_global(cond_data, cond_name )
            rejected_global = [cond_idx[i] for i in rejected]
            all_rejected.extend(rejected_global)
            print(f"Subject {sub}, Session {ses:02d} - Rejected {len(rejected)} epochs in {cond_name}")

        # Drop all rejected
        epochs_cleaned.drop(all_rejected)

        # === ROI definitions ===
        roi_definitions = {
            "Noise_ROI": ["S10_D11 hbo", "S5_D3 hbo", "S11_D11 hbo", "S10_D9 hbo", "S10_D12 hbo", "S10_D10 hbo"],
            "Speech_ROI": ["S10_D11 hbo", "S10_D10 hbo", "S10_D12 hbo", "S11_D12 hbo", "S1_D1 hbo", "S10_D9 hbo", "S8_D7 hbo", "S7_D7 hbo", "S8_D8 hbo"],
            "Common_ROI": ["S10_D11 hbo", "S10_D9 hbo", "S10_D12 hbo", "S10_D10 hbo"],
            "Only_Noise_ROI": ["S5_D3 hbo", "S11_D11 hbo"],
            "Only_Speech_ROI": ["S11_D12 hbo", "S1_D1 hbo", "S8_D7 hbo", "S7_D7 hbo", "S8_D8 hbo"],
            "Left_Auditory": ["S4_D2 hbo", "S4_D3 hbo", "S5_D2 hbo", "S5_D3 hbo", "S5_D4 hbo", "S5_D5 hbo"],
            "Right_Auditory": ["S10_D9 hbo", "S10_D10 hbo", "S10_D11 hbo", "S10_D12 hbo", "S11_D11 hbo", "S11_D12 hbo"],
            "Visual": ["S6_D6 hbo", "S6_D8 hbo", "S7_D6 hbo", "S7_D7 hbo", "S8_D7 hbo", "S8_D8 hbo", "S9_D8 hbo", "S7_D8 hbo"],
            "Front": ["S1_D1 hbo", "S2_D1 hbo", "S3_D1 hbo", "S3_D2 hbo", "S12_D1 hbo"]
        }

        tmin, tmax = 4, 6
        for roi_name, optode_list in roi_definitions.items():
            available_optodes = [opt for opt in optode_list if opt in epochs_cleaned.info["ch_names"]]
            if len(available_optodes) < 2:
                continue

            try:
                cond_data = {}
                for cond in ["Control", "Noise", "Speech"]:
                    cond_epochs = epochs_cleaned[cond].copy().pick(available_optodes).crop(tmin=tmin, tmax=tmax)
                    data = cond_epochs.get_data()
                    cond_data[cond] = data.mean(axis=1).mean(axis=1) * 1e6

                if len(cond_data["Control"]) > 1 and len(cond_data["Noise"]) > 1:
                    t_stat, p_val = ttest_ind(cond_data["Control"], cond_data["Noise"], equal_var=False, alternative='less')
                    effect = np.mean(cond_data["Noise"]) - np.mean(cond_data["Control"])
                    ttest_results.append({"Subject": sub, "Session": f"{ses:02d}", "ROI": roi_name, "Comparison": "Control vs Noise", "t_stat": t_stat, "p_value": p_val, "Effect": effect})

                if len(cond_data["Control"]) > 1 and len(cond_data["Speech"]) > 1:
                    t_stat, p_val = ttest_ind(cond_data["Control"], cond_data["Speech"], equal_var=False, alternative='less')
                    effect = np.mean(cond_data["Speech"]) - np.mean(cond_data["Control"])
                    ttest_results.append({"Subject": sub, "Session": f"{ses:02d}", "ROI": roi_name, "Comparison": "Control vs Speech", "t_stat": t_stat, "p_value": p_val, "Effect": effect})

            except Exception as e:
                print(f"Error testing {roi_name} for Subject {sub}, Session {ses:02d}: {e}")

# === Save results ===
df_ttest = pd.DataFrame(ttest_results)

# === Summary per participant ===
print("\n=== Significant ROI Summary per Participant ===")
df_significant = df_ttest[df_ttest["p_value"] < 0.05]
if df_significant.empty:
    print("No significant ROIs found.")
else:
    grouped = df_significant.groupby(["Subject", "Session"])
    for (sub, ses), group in grouped:
        print(f"\nSubject: {sub}, Session: {ses}")
        for _, row in group.iterrows():
            sign = "↑" if row["Effect"] > 0 else "↓"
            print(f"  - {row['ROI']} ({row['Comparison']}): p = {row['p_value']:.4f}, effect = {row['Effect']:.4f} ({sign})")

print("\n✅ Done with ROI t-tests!") 

Detected subjects: ['01', '02', '03', '04', '05', '07', '08', '10', '11', '12', '13', '16', '17', '19', '21', '24']
Processing Subject 01, Session 01
Subject 01, Session 01 - Rejected 2 epochs in Control
Subject 01, Session 01 - Rejected 1 epochs in Noise
Subject 01, Session 01 - Rejected 1 epochs in Speech
Processing Subject 01, Session 02
Subject 01, Session 02 - Rejected 2 epochs in Control
Subject 01, Session 02 - Rejected 1 epochs in Noise
Subject 01, Session 02 - Rejected 0 epochs in Speech
Processing Subject 02, Session 01
Subject 02, Session 01 - Rejected 5 epochs in Control
Subject 02, Session 01 - Rejected 7 epochs in Noise
Subject 02, Session 01 - Rejected 4 epochs in Speech
Processing Subject 02, Session 02
Subject 02, Session 02 - Rejected 2 epochs in Control
Subject 02, Session 02 - Rejected 3 epochs in Noise
Subject 02, Session 02 - Rejected 5 epochs in Speech
Processing Subject 03, Session 01
Subject 03, Session 01 - Rejected 6 epochs in Control
Subject 03, Session 01 -